In [5]:
import base64
import pandas as pd
from serpapi import GoogleSearch
from google.cloud import bigquery
import datetime
import time
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import dotenv
import os
dotenv.load_dotenv(dotenv.find_dotenv())

def hello_pubsub():
    
    search_term = "Analista de dados Junior"
    search_location = "Brazil"

    for num in range(45):

        start = num * 10
        params = {
            "api_key": os.getenv("API_KEY"), #Fill in with your API key from SerpApi
            "device": "desktop",
            "engine": "google_jobs",
            "google_domain": "google.com",
            "q": search_term,
            "hl": "pt",
            "gl": "br",
            "location": search_location,
            #"chips": "date_posted:today",
            "start": start,
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        # check if the last search page (i.e., no results)
        try:
            if results['error'] == "Google hasn't returned any results for this query.":
                break
        except KeyError:
            print(f"Getting SerpAPI data for page: {start}")
        else:
            continue

        # create dataframe of 10 pulled results
        jobs = results['jobs_results']
        jobs = pd.DataFrame(jobs)
        jobs = pd.concat([pd.DataFrame(jobs), 
                        pd.json_normalize(jobs['detected_extensions'])], 
                        axis=1).drop('detected_extensions', 1)
        jobs['date_time'] = datetime.datetime.utcnow()

        
        # concat dataframe
        if start == 0:
            jobs_all = jobs
        else:
            jobs_all = pd.concat([jobs_all, jobs])

        jobs_all['search_term'] = search_term
        jobs_all['search_location'] = search_location

    # table_id = "lucky-processor-339305.gsearch_job_listings.g_search" # BigQuery Table name
    # client = bigquery.Client()
    # table = client.get_table(table_id)
    # errors = client.insert_rows_from_dataframe(table, jobs_all)


    driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

    driver.get('https://www.glassdoor.com.br/Sal%C3%A1rios/index.htm');

    time.sleep(0.5)

    #Buscando o salario daquele trabalho
    driver.find_element(By.ID, 'KeywordSearch').send_keys(search_term)
    driver.find_element(By.ID,"HeroSearchButton").click()
    time.sleep(2)

    #Pegando o salario e qtd
    mediaSalarial = driver.find_element(By.CLASS_NAME, 'ebrouyy2').text
    qtdSalarios = driver.find_element(By.CLASS_NAME, 'css-1vkj9it').text
    table = {"Media salárial":[mediaSalarial], "Quantidade de Salario Reportado":[qtdSalarios]}
    testedf = pd.DataFrame(table)
    testedf.to_excel("salarios.xlsx")

    time.sleep(5)

    driver.quit()

    jobs_all['k'] = 1
    testedf['k'] = 1
    resultado = pd.merge(jobs_all, testedf, on=['k'])
    resultado = resultado.drop(["k"],axis=1)

    resultado.to_excel("\export\jobs.xlsx")
hello_pubsub()

https://serpapi.com/search
Getting SerpAPI data for page: 0
https://serpapi.com/search


C:\Users\gabri\AppData\Local\Temp\ipykernel_5304\22124498.py:55: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  axis=1).drop('detected_extensions', 1)


Getting SerpAPI data for page: 10
https://serpapi.com/search
Getting SerpAPI data for page: 20
https://serpapi.com/search


C:\Users\gabri\AppData\Local\Temp\ipykernel_5304\22124498.py:55: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  axis=1).drop('detected_extensions', 1)
C:\Users\gabri\AppData\Local\Temp\ipykernel_5304\22124498.py:55: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  axis=1).drop('detected_extensions', 1)
